# Instructions

Ce notebook ne doit contenir que votre script servant à utiliser vos modèles **entrainés** en les testant sur le dataset *fake-test.csv*. Nous devons pouvoir l'exécuter en cliquant sur *Exécution -> Tout exécuter*.

Nous utiliserons ce script pour évaluer votre modèle sur nos propres données d'évaluation par la suite.


In [3]:
from transformers import AutoModelForSequenceClassification,Trainer, AutoTokenizer
import torch
from datasets import load_dataset
import numpy as np
import evaluate

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("saved_model").cuda()
tokenizer = AutoTokenizer.from_pretrained("saved_model")
ds_evaluate = load_dataset("csv", data_files="fake_test.csv", split='train')

In [4]:
def preprocess_function(examples):
    return tokenizer(examples['data'], truncation=True, padding=True, max_length=512)

accuracy = evaluate.load("accuracy")
tokenized_evaluate = ds_evaluate.map(preprocess_function, batched=True)


trainer = Trainer(model= model)
predictions = trainer.predict(tokenized_evaluate)
predicted_labels = np.argmax(predictions.predictions, axis=1)

true_labels = ds_evaluate['label']
results = accuracy.compute(predictions=predicted_labels, references=true_labels)
print("Evaluation Results:", results)

100%|██████████| 61/61 [00:07<00:00,  7.90it/s]

Evaluation Results: {'accuracy': 0.9588477366255144}
